# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 전체 데이터 eda

In [ ]:
jemu_df = pd.read_csv('../../data/재무정보_final.csv', encoding='cp949')
jemu_df.tail()

## DF 형태 파악

In [ ]:
# 고유 기업명만
jemu_df["기업명"].unique()

In [ ]:
# 특정 기업에 대한 행 보기
jemu_df[jemu_df["기업명"] == "실리콘아츠"]

### 기업별 행 개수
- 모든 값이 다 6이 나와야 정상

In [ ]:
pd.set_option("display.max_rows", None) # 모든 행 보기
jemu_df.groupby("기업명").size().sort_values(ascending=False)

### 기업별 결측치 비율
- 1에 가까울수록 거의 다 결측

In [ ]:
company_missing_ratio = (
    jemu_df
    .groupby("기업명")
    .apply(lambda x: x.isna().mean())
)

company_missing_ratio.head()

## 변수 기준으로 기업별 분포 보기
- (우선) 변수 기준 : 매출액
-> 이렇게 하면 빈 값 확인 가능

In [ ]:
pd.set_option("display.max_rows", None) # 모든 행 보기
jemu_df.groupby("기업명")["매출액"].count().sort_values()